In [6]:
import pandas as pd
from pathlib import Path

RAW = Path("../data/raw")

df_data = pd.read_csv(RAW / "ethiopia_fi_unified_data.csv", dtype=str)

df_ref  = pd.read_csv(RAW/ "reference_codes.csv", dtype=str)
df_link = pd.read_csv(RAW/ "impact_links.csv", dtype=str)

for c in ["observation_date", "period_start", "period_end"]:
    if c in df_data.columns:
        df_data[c] = pd.to_datetime(df_data[c], errors="coerce")
if "observation_date" in df_link.columns:
    df_link["observation_date"] = pd.to_datetime(df_link["observation_date"], errors="coerce")

print("record_type counts:\n", df_data["record_type"].value_counts(dropna=False))
print("\npillar counts:\n", df_data["pillar"].fillna("<<NA>>").value_counts())
print("\nconfidence distribution:\n", df_data["confidence"].fillna("<<NA>>").value_counts())
print("\nObservation date range:",
      df_data["observation_date"].min(), "→", df_data["observation_date"].max())

obs = df_data[df_data["record_type"]=="observation"].copy()
if not obs.empty:
    obs["year"]=obs["observation_date"].dt.year
    cov = obs.groupby("indicator_code")["year"].agg(["min","max","count"])
    print("\nIndicator coverage:\n", cov.head(30))

ev = df_data[df_data["record_type"]=="event"].sort_values("observation_date")
print("\nEvents (id,category,date):")
print(ev[["record_id","category","indicator","observation_date"]].head(20))

print("\nImpact link sample:\n", df_link.head())


record_type counts:
 observation    30
event          10
target          3
Name: record_type, dtype: int64

pillar counts:
 ACCESS           16
USAGE            11
<<NA>>           10
GENDER            5
AFFORDABILITY     1
Name: pillar, dtype: int64

confidence distribution:
 high      40
medium     3
Name: confidence, dtype: int64

Observation date range: 2014-12-31 00:00:00 → 2030-12-31 00:00:00

Indicator coverage:
                      min   max  count
indicator_code                       
ACC_4G_COV          2023  2025      2
ACC_FAYDA           2024  2025      3
ACC_MM_ACCOUNT      2021  2024      2
ACC_MOBILE_PEN      2025  2025      1
ACC_OWNERSHIP       2014  2024      6
AFF_DATA_INCOME     2024  2024      1
GEN_GAP_ACC         2021  2024      2
GEN_GAP_MOBILE      2024  2024      1
GEN_MM_SHARE        2024  2024      1
USG_ACTIVE_RATE     2024  2024      1
USG_ATM_COUNT       2025  2025      1
USG_ATM_VALUE       2025  2025      1
USG_CROSSOVER       2025  2025      1
USG_MP

In [9]:
errors = {}
errors["duplicate_ids"] = df_data["record_id"].duplicated().sum()
errors["obs_missing_date"] = df_data.loc[df_data["record_type"]=="observation", "observation_date"].isna().sum()
errors["event_has_pillar"] = df_data.loc[df_data["record_type"]=="event", "pillar"].notna().sum()

unmatched_links = ~df_link["parent_id"].isin(df_data.loc[df_data["record_type"]=="event", "record_id"])
errors["impactlinks_bad_parent"] = unmatched_links.sum()

print(errors)


{'duplicate_ids': 0, 'obs_missing_date': 0, 'event_has_pillar': 0, 'impactlinks_bad_parent': 0}
